In [ ]:
import copy

# Define the crossword board and words
words = ["PINEAPPLE", "BLUEBERRY", "PLUM", "PEAR", "KIWI", "MANGO", "MELON", "LEMON"]
grid = [
    ['1', '.', '.', '.', '#', '2', '#', '#', '3'],
    ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
    ['#', '4', '.', '5', '.', '.', '#', '#', '.'],
    ['6', '#', '#', '.', '#', '.', '#', '#', '.'],
    ['.', '#', '#', '.', '#', '.', '#', '#', '.'],
    ['.', '#', '#', '.', '#', '.', '#', '7', '#'],
    ['.', '#', '#', '.', '#', '.', '#', '.', '#'],
    ['#', '#', '#', '#', '#', '.', '#', '.', '#'],
    ['8', '.', '.', '.', '.', '.', '.', '.', '.']
]

player_placed = []
computer_placed = []

# Mapping grid numbers to positions and directions and solution word (predefined)
grid_mapping = {
    '1': ([0, 0], [0,3], 'horizontal',  ["PLUM", "KIWI"]),
    '2': ([0, 5], [8,5], 'vertical',    ["PINEAPPLE"]),
    '3': ([0, 8], [4,8], 'vertical',    ["LEMON", "MELON", "MANGO"]),
    '4': ([2, 1], [2,5], 'horizontal',  ["LEMON", "MELON"]),
    '5': ([2, 3], [6,3], 'vertical' ,   ["MELON","LEMON", "MANGO"]),
    '6': ([3, 0], [6,0], 'vertical',    ["PLUM", "KIWI"]),
    '7': ([5, 7], [8,7], 'vertical',    ["PEAR"]),
    '8': ([8, 0], [8,8], 'horizontal',  ["BLUEBERRY"])
}

# Functions for placing and removing words from the grid
def place(word, position, grid):
    start_pos, end_pos, direction, possible_words  = position
    new_grid = copy.deepcopy(grid)
    if direction == 'horizontal':
        for i, letter in enumerate(word):
            new_grid[start_pos[0]][start_pos[1] + i] = letter
    elif direction == 'vertical':
        for i, letter in enumerate(word):
            new_grid[start_pos[0]+ i][start_pos[1]] = letter
    return new_grid

# Function to check if a word can be placed at a position
def can_place(word, start_pos, end_pos, direction, possible_words, grid):
    #print(f"checking can place with state {word}  AND  {start_pos}  AND {end_pos} AND  {direction} AND {possible_words}")
    if direction == 'horizontal':
        #print(f" {direction}  start_pos[1] > {start_pos[1]} AND end_pos[1] > {end_pos[1]}")
        if end_pos[1] - start_pos[1] + 1 != len(word):  # Check if word fits in the row horizontally
            return False
        if word not in possible_words:
            return False
        for i in range(len(word)):
            if grid[start_pos[0]][start_pos[1] + i] not in ['.', word[i], '1', '2', '3', '4', '5', '6', '7', '8']:  # Ensure space is free or matches the word
                return False
    elif direction == 'vertical':
        #print(f" {direction}  start_pos[0] > {start_pos[0]} AND end_pos[0] > {end_pos[0]}")
        if  end_pos[0] - start_pos[0] + 1 != len(word):  # Check if word fits in the column vertically
            return False
        if word not in possible_words:
            return False
        for i in range(len(word)):
            if grid[start_pos[0]+ i][start_pos[1]] not in ['.', word[i], '1', '2', '3', '4', '5', '6', '7', '8']:  # Ensure space is free or matches the word
                return False
    return True

# Function to generate valid moves (only basic horizontal/vertical for simplicity)
def generate_moves(state):
    words, grid, positions, placed, player_score, computer_score, turn = state
    valid_moves = []
    for word in words:
        if word not in placed:
            for key, position in grid_mapping.items():
                start_pos, end_pos, direction, possible_words = position
                if can_place(word, start_pos, end_pos, direction, possible_words, grid):
                    valid_moves.append((word, (start_pos, end_pos, direction,possible_words)))
    return valid_moves

# Evaluation function (more words placed = better score)
def evaluateword(word):
    return len(word)

def evaluate(placed):
    return len(placed)


def getscore(player_placed):
    sum = 0
    for word in player_placed:
        sum = sum + len(word)
    return sum


def static_evaluation(state):
    words, grid, positions, placed, player_score, computer_score, turn = state
    # For simplicity, return the difference in scores as an evaluation
    print(f"in evaluation function  Player : {player_score} | Computer : {computer_score}")
    return player_score - computer_score

# Function to check if the game is over
def is_terminal(state):
    words, grid, positions, placed, player_score, computer_score, turn = state
    
    # Game is over if all words are placed
    if len(placed) == len(words):
        print("chcking here is game over")
        return True

    # Game is over if no valid moves remain for the current player
    if len(generate_moves(state)) == 0:
        print("checking generate_moves if game over")
        return True

    return False

# Min-Max algorithm with alpha-beta pruning
def min_max(state, depth, alpha, beta):
    words, grid, positions, placed, player_score, computer_score, turn = state
    if is_terminal(state) or depth == 0:
        return getscore(placed), None

    moves = generate_moves(state)
    best_move = None

    if turn == "Player1":
        best_score = -float("inf")
        for move in moves:
            word, position = move
            new_grid = place(word, position, grid)
            new_state = (words, new_grid, positions, placed + [word], player_score,computer_score, get_opponent(turn))
            score, _ = min_max(new_state, depth - 1, alpha, beta)
            if score > best_score:
                best_score = score
                best_move = move
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
    else:
        best_score = float("inf")
        for move in moves:
            word, position = move
            new_grid = place(word, position, grid)
            new_state = (words, new_grid, positions, placed + [word],player_score,computer_score, get_opponent(turn))
            score, _ = min_max(new_state, depth - 1, alpha, beta)
            if score < best_score:
                best_score = score
                best_move = move
            beta = min(beta, best_score)
            if beta <= alpha:
                break

    return best_score, best_move

# Function to switch turns
def get_opponent(turn):
    return "Player2" if turn == "Player1" else "Player1"

# Function to print the grid
def print_grid(grid):
    for row in grid:
        print(' '.join(row))
    print()

def blockword(start_pos, end_pos, direction, possible_words , word):
    possible_words.remove(word)
    new_grid_row = (start_pos, end_pos, direction, possible_words)
    return new_grid_row

# Function to play the game
def play_game():
    global grid  # Declare `grid` as global to avoid UnboundLocalError
    global player_placed
    global computer_placed
    global grid_mapping
    current_grid = copy.deepcopy(grid)  # Now it properly references the global `grid`
    state = (words, current_grid, [], [], 0, 0, "Player1")

    while not is_terminal(state):
        _, grid, positions, placed, player_score, computer_score, turn = state

        if turn == "Player1":
            print("========================================new Turn============================================")
            print("Current Grid:")
            print_grid(grid)
            print(f"Words placed: {placed}")
            print("Available positions: ", list(grid_mapping.keys()))
            print("Your turn. Available words:", [w for w in words if w not in placed])
            
            # Input from the user: grid number (1, 2, 3...) and word
            selected_number = input("Enter the number where you want to place the word: ")
            if selected_number in grid_mapping:
                word = input("Enter the word you want to place: ").upper()
                position = grid_mapping[selected_number]
                
                if word in words and word not in placed and can_place(word, *position, grid):
                    grid = place(word, position, grid)
                    grid_mapping[selected_number] = blockword(*position, word)
                    placed.append(word)
                    player_placed.append(word)
                    player_score = player_score + evaluateword(word)
                    state = (words, grid, positions, placed, player_score,computer_score, get_opponent(turn))
                    score = static_evaluation(state) ## Calling static evaluation function
                    del grid_mapping[selected_number]
                else:
                    print("Invalid move!!")
                    player_score = player_score - 1
                    state = (words, grid, positions, placed, player_score,computer_score, get_opponent(turn))
                    score = static_evaluation(state)
            else:
                print("Invalid position")
                player_score = player_score - 1
                state = (words, grid, positions, placed, player_score,computer_score, get_opponent(turn))
                score = static_evaluation(state)
        
        else:  # Player2 (Computer)
            print("Computer's turn...")
            _, best_move = min_max(state, 3, -float("inf"), float("inf"))
            if best_move:
                word, position = best_move
                print(f"Computer places {word} at {position[0]}")
                grid = place(word, position, grid)
                placed.append(word)
                computer_placed.append(word)
                computer_score = computer_score + evaluateword(word)  # Update score after the word is placed
                state = (words, grid, positions, placed, player_score, computer_score, get_opponent(turn))
                score = static_evaluation(state) ## Calling static evaluation function
                grid_mapping = {key: val for key,val in grid_mapping.items() if val != position }
                

    print("Game over! Final Grid:")
    print_grid(state[1])
    print(f"Words placed: {state[3]}")
    final_score = static_evaluation(state)
    print(f"Final Score: {final_score}")

# Run the game
play_game()


========================================new Turn============================================
Current Grid:
1 . . . # 2 # # 3
# # # # # . # # .
# 4 . 5 . . # # .
6 # # . # . # # .
. # # . # . # # .
. # # . # . # 7 #
. # # . # . # . #
# # # # # . # . #
8 . . . . . . . .

Words placed: []
Available positions:  ['1', '2', '3', '4', '5', '6', '7', '8']
Your turn. Available words: ['PINEAPPLE', 'BLUEBERRY', 'PLUM', 'PEAR', 'KIWI', 'MANGO', 'MELON', 'LEMON']


Enter the number where you want to place the word:  7
Enter the word you want to place:  PEAR


in evaluation function  Player : 4 | Computer : 0
Computer's turn...
Computer places PLUM at [0, 0]
in evaluation function  Player : 4 | Computer : 4
========================================new Turn============================================
Current Grid:
P L U M # 2 # # 3
# # # # # . # # .
# 4 . 5 . . # # .
6 # # . # . # # .
. # # . # . # # .
. # # . # . # P #
. # # . # . # E #
# # # # # . # A #
8 . . . . . . R .

Words placed: ['PEAR', 'PLUM']
Available positions:  ['2', '3', '4', '5', '6', '8']
Your turn. Available words: ['PINEAPPLE', 'BLUEBERRY', 'KIWI', 'MANGO', 'MELON', 'LEMON']


Enter the number where you want to place the word:  1


Invalid position
in evaluation function  Player : 3 | Computer : 4
Computer's turn...
Computer places KIWI at [3, 0]
in evaluation function  Player : 3 | Computer : 8
========================================new Turn============================================
Current Grid:
P L U M # 2 # # 3
# # # # # . # # .
# 4 . 5 . . # # .
K # # . # . # # .
I # # . # . # # .
W # # . # . # P #
I # # . # . # E #
# # # # # . # A #
8 . . . . . . R .

Words placed: ['PEAR', 'PLUM', 'KIWI']
Available positions:  ['2', '3', '4', '5', '8']
Your turn. Available words: ['PINEAPPLE', 'BLUEBERRY', 'MANGO', 'MELON', 'LEMON']


Enter the number where you want to place the word:  BLUEBERRY


Invalid position
in evaluation function  Player : 2 | Computer : 8
Computer's turn...
Computer places MANGO at [0, 8]
in evaluation function  Player : 2 | Computer : 13
========================================new Turn============================================
Current Grid:
P L U M # 2 # # M
# # # # # . # # A
# 4 . 5 . . # # N
K # # . # . # # G
I # # . # . # # O
W # # . # . # P #
I # # . # . # E #
# # # # # . # A #
8 . . . . . . R .

Words placed: ['PEAR', 'PLUM', 'KIWI', 'MANGO']
Available positions:  ['2', '4', '5', '8']
Your turn. Available words: ['PINEAPPLE', 'BLUEBERRY', 'MELON', 'LEMON']


Enter the number where you want to place the word:  5
Enter the word you want to place:  MELON


in evaluation function  Player : 7 | Computer : 13
Computer's turn...
chcking here is game over
chcking here is game over
chcking here is game over
chcking here is game over
Computer places PINEAPPLE at [0, 5]
in evaluation function  Player : 7 | Computer : 22
========================================new Turn============================================
Current Grid:
P L U M # P # # M
# # # # # I # # A
# 4 . M . N # # N
K # # E # E # # G
I # # L # A # # O
W # # O # P # P #
I # # N # P # E #
# # # # # L # A #
8 . . . . E . R .

Words placed: ['PEAR', 'PLUM', 'KIWI', 'MANGO', 'MELON', 'PINEAPPLE']
Available positions:  ['4', '8']
Your turn. Available words: ['BLUEBERRY', 'LEMON']
